# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
1. Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
2. Import Yelp business data and join to inspection data    
4. Join the Yelp business and Inspection Data with the Yelp Review Data
    + Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    +  Aggregate any review "count" features using this same logic
    + Combine the review documents for a restaurant into a CLOB using the same logic
    
### TO DO:
1. Determine how we want to do Levenshtein matching to combine the datasets (see the *Join Yelp Review Data with Inspection Dataset* below) 
2. Create additional engineered features
3. n-gram extraction
4. vectorization of n-grams
5. model selection
    + training
    + validaiton
    + evaluation
    +repeat with additional or removed features and data segmentation (if aggregating the review text does not yield successful results)

## Import and Clean Data

In [1]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np
import ast

In [2]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()

In [3]:
#cur.execute("SELECT * FROM public.restaurants LIMIT 50 ")
#biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_checkins")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_reviews")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [4]:
reviews.columns = ['bizID','reviewID','userID','type','stars','text','useful','funny','cool','date']
#get total reviews per biz
rev = reviews['bizID'].value_counts()
rev_counts = pd.DataFrame(rev).reset_index()
rev_counts.columns = ['bizID','all_review_count']

In [5]:
#not using this - can delete

checkins.columns = ['bizID','type','datetime']
#get total checkins per biz
chks  = checkins['bizID'].value_counts()
chk_counts = pd.DataFrame(chks).reset_index()
chk_counts.columns = ['bizID','checkin_counts']

### Join Yelp Review Data with Inspection Dataset

We have a few options here. While it is optimal to have as much done in Python as possible, the matching process in python is impractically slow. We can 1) use a manual implementation of Levenshtein Distance (LD), 2) use a package with optimized LD code, or 3) do the joining of the inspeciton data and review data in our database. OPtions 1& 2 are shown below, and code to retrieve the results of option 3 are at the bottom of this section.

If we decide to go with option 3, the last steps are to combine all records where the business ID, last inspection date, and inspection date are equal in order to get to one observation per restaurant-inspection combination. We can then add in any other engineered features.

**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

#### Levenshtein Option #3 (in-database) 
This option joins the yelp restaurant informaiton to each inspection record where:
 * The Levenshetein distance of the restaurant name from the two datasets is <3
 * The distance of the address from each dataset is <4
 * The date of the review is greater than the prior inspection date
 * The date of the review is less than or equal to inspeciton date on the record
 
Whitespace at the beginning and end of the name and address in each dataset is trimmed, and the strings are converted to uppercase before matching. The mathcing thresholds can be adjusted to increase potential for matching, or decrease false matches.

In [6]:
# The materialized view of the restaurant, inspection, and review data is "toronto_all"
cur.execute("SELECT * FROM public.toronto_all_2 where review_date is not null and attributes is not null" )
obs = pd.DataFrame(cur.fetchall())
obs.head()
obs.columns=['bizID','name','address','postal_code','neighborhood','lat','long','categories','attributes','is_open','review_cnt','hours','stars','setablishment_id','establishment_name','establishment_address','inspection_date','last_inspection','count_minor','count_sig','count_crucial','count_na','count_crucial_signficant','review_id','user_id','review_stars','review_text','useful','funny','cool','review_dt']
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,count_na,count_crucial_signficant,review_id,user_id,review_stars,review_text,useful,funny,cool,review_dt
0,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,k1YwiLwOuvdd-mRfqTjOuQ,YKU5vy0dWFP1MLexzzVLQg,3,"""Good food, lovely service. A little loud for ...",0,0,0,2015-08-15
1,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,1,AZyLvCxyN5DnhyY2JboEgA,fSYQ_oQ0wNGk2nMJLvNZJA,1,"""Ok so imagine that is Halloween all year arou...",0,0,0,2016-12-19
2,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,0Y3ksUrPhSjdFwgYXgXPlw,2MA4jXpayOxqzul0e6yPDg,4,"""Great little cheap dive bar with a dimly lit ...",1,0,0,2016-06-15
3,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,dN3IVxSXlUC8kwfCYt155w,BKF78sXmNHReyj3YTGfHtQ,4,A wonderful wonderful neighborhood bar with a ...,0,0,0,2016-08-15
4,0ORDYMDX027EhfsoFOkNcQ,BRITISHSTYLEFISHCHIPS,73COXWELLAVENUE,M4L 3B1,Upper Beach,43.6686238,-79.3172737,"['Restaurants', 'Fish & Chips']","['Alcohol: none', ""Ambience: {'romantic': Fals...",1,...,0,0,pgEUxcu0fGb7jlsC3h7zhg,VP7_Jz3wdklXvbfOHiBLLA,5,Love this place. They have the best pub style ...,0,0,0,2016-02-07


In [7]:
obs['bizID-dt'] = obs['bizID'] + "-" + obs['inspection_date'].map(str)

In [8]:
in_scope_rev = obs['bizID-dt'].value_counts()
in_scope_reviews = pd.DataFrame(in_scope_rev).reset_index()
in_scope_reviews.columns = ['bizID-dt','count_reviews_in_scope']
in_scope_reviews.head()

,bizID-dt,count_reviews_in_scope
0,RwRNR4z3kY-4OsFqigY5sw-2015-11-03,248
1,jc3p5SFyt9qrrMXt6E13ig-2016-12-21,161
2,RwRNR4z3kY-4OsFqigY5sw-2016-03-15,153
3,h_4dPV9M9aYaBliH1Eoeeg-2016-05-12,141
4,trKyIRyjKqVSZmcU0AnICQ-2016-04-11,114


In [9]:
#get dummies for star rating column
obs = pd.concat([obs, pd.get_dummies(obs['review_stars'], prefix='stars')], axis=1)
obs.head()

,bizID,name,address,postal_code,neighborhood,lat,long,categories,attributes,is_open,...,useful,funny,cool,review_dt,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,0,2015-08-15,0o9imYHyCGVvkR0NyJDCoQ-2016-03-11,0,0,1,0,0
1,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,0,2016-12-19,0o9imYHyCGVvkR0NyJDCoQ-2017-01-30,1,0,0,0,0
2,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,1,0,0,2016-06-15,0o9imYHyCGVvkR0NyJDCoQ-2016-09-12,0,0,0,1,0
3,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,992QUEENSTREETW,M6J 1H1,Ossington Strip,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,0,0,0,2016-08-15,0o9imYHyCGVvkR0NyJDCoQ-2016-09-12,0,0,0,1,0
4,0ORDYMDX027EhfsoFOkNcQ,BRITISHSTYLEFISHCHIPS,73COXWELLAVENUE,M4L 3B1,Upper Beach,43.6686238,-79.3172737,"['Restaurants', 'Fish & Chips']","['Alcohol: none', ""Ambience: {'romantic': Fals...",1,...,0,0,0,2016-02-07,0ORDYMDX027EhfsoFOkNcQ-2016-05-10,0,0,0,0,1


In [10]:
stars = obs.groupby('bizID-dt')[['stars_1', 'stars_2','stars_3','stars_4','stars_5']].sum().reset_index()
stars.head()

,bizID-dt,stars_1,stars_2,stars_3,stars_4,stars_5
0,-2TBP3ZGu7M-FmfoNJvbrQ-2016-09-07,0,1,0,2,0
1,-2TBP3ZGu7M-FmfoNJvbrQ-2017-01-18,1,0,1,3,1
2,-76didnxGiiMO80BjSpYsQ-2015-09-01,1,1,0,0,0
3,-76didnxGiiMO80BjSpYsQ-2016-03-01,1,4,0,1,0
4,-76didnxGiiMO80BjSpYsQ-2016-09-29,2,3,0,1,1


In [11]:
combined_revs = obs.groupby('bizID-dt')['review_text'].apply(' '.join).reset_index()
combined_revs.head()

,bizID-dt,review_text
0,-2TBP3ZGu7M-FmfoNJvbrQ-2016-09-07,"""I loveee bacon, and I gotta say, Rashers did ..."
1,-2TBP3ZGu7M-FmfoNJvbrQ-2017-01-18,I like this place 100%. When what you want is ...
2,-76didnxGiiMO80BjSpYsQ-2015-09-01,I used to frequent this place but stopped afte...
3,-76didnxGiiMO80BjSpYsQ-2016-03-01,"great location, good and relaxing interior set..."
4,-76didnxGiiMO80BjSpYsQ-2016-09-29,"""Avoid if possible. The regular coffee is too ..."


In [12]:
users = obs.groupby('bizID-dt')['user_id'].count().reset_index()
users.columns = ['bizID-dt','count_unique_users']
users.head()

,bizID-dt,count_unique_users
0,-2TBP3ZGu7M-FmfoNJvbrQ-2016-09-07,3
1,-2TBP3ZGu7M-FmfoNJvbrQ-2017-01-18,6
2,-76didnxGiiMO80BjSpYsQ-2015-09-01,2
3,-76didnxGiiMO80BjSpYsQ-2016-03-01,6
4,-76didnxGiiMO80BjSpYsQ-2016-09-29,7


In [13]:
sub = obs[['bizID-dt','bizID','name','postal_code','lat','long','categories','attributes','is_open','count_crucial_signficant']]
sub = sub.drop_duplicates()
sub.head()

,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open,count_crucial_signficant
0,0o9imYHyCGVvkR0NyJDCoQ-2016-03-11,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,M6J 1H1,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,0
1,0o9imYHyCGVvkR0NyJDCoQ-2017-01-30,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,M6J 1H1,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,1
2,0o9imYHyCGVvkR0NyJDCoQ-2016-09-12,0o9imYHyCGVvkR0NyJDCoQ,LIPSTICKDYNAMITE,M6J 1H1,43.6444125,-79.4183518,"['Vegetarian', 'Cocktail Bars', 'Nightlife', '...","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,0
4,0ORDYMDX027EhfsoFOkNcQ-2016-05-10,0ORDYMDX027EhfsoFOkNcQ,BRITISHSTYLEFISHCHIPS,M4L 3B1,43.6686238,-79.3172737,"['Restaurants', 'Fish & Chips']","['Alcohol: none', ""Ambience: {'romantic': Fals...",1,0
8,0ORDYMDX027EhfsoFOkNcQ-2016-12-22,0ORDYMDX027EhfsoFOkNcQ,BRITISHSTYLEFISHCHIPS,M4L 3B1,43.6686238,-79.3172737,"['Restaurants', 'Fish & Chips']","['Alcohol: none', ""Ambience: {'romantic': Fals...",1,0


In [138]:
#merge in all data into one df
df1 = pd.merge(sub,stars,on='bizID-dt', how='left')
df2 = pd.merge(df1,combined_revs,on='bizID-dt', how='left')
df3 = pd.merge(df2,rev_counts,on='bizID', how='left')
df4 = pd.merge(df3,in_scope_reviews,on='bizID-dt', how='left')
df5 = pd.merge(df4,users,on='bizID-dt', how='left')


In [139]:
t = []
for i in range(len(df5['categories'])):
    x = ast.literal_eval(df5['categories'][i])
    t.append(x)
    
cats = pd.DataFrame(t)
cats_df = pd.get_dummies(cats, prefix='Category')
cats_df = cats_df.groupby(cats_df.columns, axis=1).sum()


atts_df = pd.DataFrame()
for x in range(len(df5['attributes'])):
    list_yelp = ast.literal_eval(df5['attributes'][x])

    attribute_list = []
    attribute_name = []

    for i in list_yelp:

        name = i.split(":")[0]
        values = i.split( name+": " )[1].replace('{','').replace('}','')

        if len(values.split(":")) > 1: 
        
            for j in values.split(","):
                name_j = name + "_" + (j.split(":")[0].strip().replace("'",''))
                attribute_name.append( name_j )
                attribute_list.append (j.split(":")[1])
        else:
            attribute_name.append( name )
            attribute_list.append ( values )
        
    dataframe = pd.DataFrame(attribute_list).transpose()
    dataframe.columns = attribute_name
    atts_df = atts_df.append(dataframe)
    
atts_df = atts_df.reset_index().drop('index', 1)

df = pd.concat([df5, cats_df, atts_df], axis=1)

#make sure each bizID-dt is only appearing once in the data
print('Max of number of unique bizID-dt in df (should be 1):',max(df5['bizID-dt'].value_counts()))


Max of number of unique bizID-dt in df (should be 1): 2


In [140]:
df

,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,is_open,count_crucial_signficant,stars_1,stars_2,stars_3,stars_4,stars_5,review_text,all_review_count,count_reviews_in_scope,count_unique_users,Category_Active Life,Category_Afghan,Category_African,Category_American (New),Category_American (Traditional),Category_Antiques,Category_Arabian,Category_Arcades,Category_Argentine,Category_Art Classes,Category_Art Galleries,Category_Art Schools,Category_Arts & Crafts,Category_Arts & Entertainment,Category_Asian Fusion,Category_Australian,Category_Austrian,Category_Automotive,Category_Bagels,Category_Bakeries,Category_Bangladeshi,Category_Barbeque,Category_Bars,Category_Beauty & Spas,Category_Beer,Category_Beer Bar,Category_Belgian,Category_Bistros,Category_Books,Category_Brasseries,Category_Brazilian,Category_Breakfast & Brunch,Category_Breweries,Category_British,Category_Bubble Tea,Category_Buffets,Category_Burgers,Category_Butcher,Category_Cabaret,Category_Cafes,Category_Cajun/Creole,Category_Canadian (New),Category_Candy Stores,Category_Cards & Stationery,Category_Caribbean,Category_Casinos,Category_Caterers,Category_Cheese Shops,Category_Cheesesteaks,Category_Chicken Shop,Category_Chicken Wings,Category_Chinese,Category_Chocolatiers & Shops,Category_Cocktail Bars,Category_Coffee & Tea,Category_Coffee Roasteries,Category_Colombian,Category_Comedy Clubs,Category_Comfort Food,Category_Comic Books,Category_Convenience Stores,Category_Cooking Schools,Category_Creperies,Category_Cuban,Category_Cupcakes,Category_Dance Clubs,Category_Delicatessen,Category_Delis,Category_Department Stores,Category_Desserts,Category_Dim Sum,Category_Diners,Category_Dive Bars,Category_Do-It-Yourself Food,Category_Donairs,Category_Donuts,Category_Drugstores,Category_Education,Category_Egyptian,Category_Ethic Grocery,Category_Ethiopian,Category_Ethnic Food,Category_Ethnic Grocery,Category_Event Planning & Services,Category_Falafel,Category_Farmers Market,Category_Fashion,Category_Fast Food,Category_Filipino,Category_Fish & Chips,Category_Florists,Category_Flowers & Gifts,Category_Food,Category_Food Court,Category_Food Delivery Services,Category_Food Stands,Category_Food Trucks,Category_French,Category_Fruits & Veggies,Category_Gas & Service Stations,Category_Gastropubs,Category_German,Category_Gift Shops,Category_Gluten-Free,Category_Greek,Category_Grocery,Category_Hakka,Category_Halal,Category_Hawaiian,Category_Health & Medical,Category_Health Markets,Category_Himalayan/Nepalese,Category_Hookah Bars,Category_Hot Dogs,Category_Hot Pot,Category_Hotels,Category_Hotels & Travel,Category_Hungarian,Category_IT Services & Computer Repair,Category_Ice Cream & Frozen Yogurt,Category_Imported Food,Category_Indian,Category_Indonesian,Category_International,Category_International Grocery,Category_Internet Cafes,Category_Irish,Category_Italian,Category_Japanese,Category_Jazz & Blues,Category_Juice Bars & Smoothies,Category_Karaoke,Category_Kebab,Category_Korean,Category_Kosher,Category_Laotian,Category_Latin American,Category_Lebanese,Category_Live/Raw Food,Category_Local Flavor,Category_Local Services,Category_Lounges,Category_Macarons,Category_Mags,Category_Malaysian,Category_Meat Shops,Category_Mediterranean,Category_Mexican,Category_Middle Eastern,Category_Mobile Phone Repair,Category_Modern European,Category_Moroccan,Category_Mosques,Category_Music & Video,Category_Music Venues,Category_Nicaraguan,Category_Nightlife,Category_Noodles,Category_Nutritionists,Category_Organic Stores,Category_Pakistani,Category_Party & Event Planning,Category_Pasta Shops,Category_Patisserie/Cake Shop,Category_Persian/Iranian,Category_Peruvian,Category_Pizza,Category_Playgrounds,Category_Polish,Category_Pool Halls,Category_Pop-up Shops,Category_Portuguese,Category_Poutineries,Category_Pretzels,Category_Pubs,Category_Ramen,Category_Recreation Centers,Category_Religious Organizations,Category_Restaurants,Category_Russian,Category_Salad,Category_Salvadoran,Category_Sandwiches,Cat

In [142]:
#dummy-ize all categorical and boolean variables
df=pd.get_dummies(df, columns=[
 'Alcohol',
 'Ambience_casual',
 'Ambience_classy',
 'Ambience_hipster',
 'Ambience_intimate',
 'Ambience_romantic',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'BikeParking',
 'BusinessAcceptsCreditCards',
 'BusinessParking_garage',
 'BusinessParking_lot',
 'BusinessParking_street',
 'BusinessParking_valet',
 'BusinessParking_validated',
 'Caters',
 'GoodForKids',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'GoodForMeal_dessert',
 'GoodForMeal_dinner',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'HasTV',
 'NoiseLevel',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'WiFi', 
 'is_open',
'BestNights_friday',
 'BestNights_monday',
 'BestNights_saturday',
 'BestNights_sunday',
 'BestNights_thursday',
 'BestNights_tuesday',
 'BestNights_wednesday',
     'ByAppointmentOnly',
 'CoatCheck',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'HappyHour',
 'Music_background_music',
 'Music_dj',
 'Music_jukebox',
 'Music_karaoke',
 'Music_live',
 'Music_no_music',
 'Music_video',
 'Open24Hours',
 'RestaurantsCounterService',
 'Smoking',
 'WheelchairAccessible'
  ]
  , dummy_na=True)
df


,bizID-dt,bizID,name,postal_code,lat,long,categories,attributes,count_crucial_signficant,stars_1,stars_2,stars_3,stars_4,stars_5,review_text,all_review_count,count_reviews_in_scope,count_unique_users,Category_Active Life,Category_Afghan,Category_African,Category_American (New),Category_American (Traditional),Category_Antiques,Category_Arabian,Category_Arcades,Category_Argentine,Category_Art Classes,Category_Art Galleries,Category_Art Schools,Category_Arts & Crafts,Category_Arts & Entertainment,Category_Asian Fusion,Category_Australian,Category_Austrian,Category_Automotive,Category_Bagels,Category_Bakeries,Category_Bangladeshi,Category_Barbeque,Category_Bars,Category_Beauty & Spas,Category_Beer,Category_Beer Bar,Category_Belgian,Category_Bistros,Category_Books,Category_Brasseries,Category_Brazilian,Category_Breakfast & Brunch,Category_Breweries,Category_British,Category_Bubble Tea,Category_Buffets,Category_Burgers,Category_Butcher,Category_Cabaret,Category_Cafes,Category_Cajun/Creole,Category_Canadian (New),Category_Candy Stores,Category_Cards & Stationery,Category_Caribbean,Category_Casinos,Category_Caterers,Category_Cheese Shops,Category_Cheesesteaks,Category_Chicken Shop,Category_Chicken Wings,Category_Chinese,Category_Chocolatiers & Shops,Category_Cocktail Bars,Category_Coffee & Tea,Category_Coffee Roasteries,Category_Colombian,Category_Comedy Clubs,Category_Comfort Food,Category_Comic Books,Category_Convenience Stores,Category_Cooking Schools,Category_Creperies,Category_Cuban,Category_Cupcakes,Category_Dance Clubs,Category_Delicatessen,Category_Delis,Category_Department Stores,Category_Desserts,Category_Dim Sum,Category_Diners,Category_Dive Bars,Category_Do-It-Yourself Food,Category_Donairs,Category_Donuts,Category_Drugstores,Category_Education,Category_Egyptian,Category_Ethic Grocery,Category_Ethiopian,Category_Ethnic Food,Category_Ethnic Grocery,Category_Event Planning & Services,Category_Falafel,Category_Farmers Market,Category_Fashion,Category_Fast Food,Category_Filipino,Category_Fish & Chips,Category_Florists,Category_Flowers & Gifts,Category_Food,Category_Food Court,Category_Food Delivery Services,Category_Food Stands,Category_Food Trucks,Category_French,Category_Fruits & Veggies,Category_Gas & Service Stations,Category_Gastropubs,Category_German,Category_Gift Shops,Category_Gluten-Free,Category_Greek,Category_Grocery,Category_Hakka,Category_Halal,Category_Hawaiian,Category_Health & Medical,Category_Health Markets,Category_Himalayan/Nepalese,Category_Hookah Bars,Category_Hot Dogs,Category_Hot Pot,Category_Hotels,Category_Hotels & Travel,Category_Hungarian,Category_IT Services & Computer Repair,Category_Ice Cream & Frozen Yogurt,Category_Imported Food,Category_Indian,Category_Indonesian,Category_International,Category_International Grocery,Category_Internet Cafes,Category_Irish,Category_Italian,Category_Japanese,Category_Jazz & Blues,Category_Juice Bars & Smoothies,Category_Karaoke,Category_Kebab,Category_Korean,Category_Kosher,Category_Laotian,Category_Latin American,Category_Lebanese,Category_Live/Raw Food,Category_Local Flavor,Category_Local Services,Category_Lounges,Category_Macarons,Category_Mags,Category_Malaysian,Category_Meat Shops,Category_Mediterranean,Category_Mexican,Category_Middle Eastern,Category_Mobile Phone Repair,Category_Modern European,Category_Moroccan,Category_Mosques,Category_Music & Video,Category_Music Venues,Category_Nicaraguan,Category_Nightlife,Category_Noodles,Category_Nutritionists,Category_Organic Stores,Category_Pakistani,Category_Party & Event Planning,Category_Pasta Shops,Category_Patisserie/Cake Shop,Category_Persian/Iranian,Category_Peruvian,Category_Pizza,Category_Playgrounds,Category_Polish,Category_Pool Halls,Category_Pop-up Shops,Category_Portuguese,Category_Poutineries,Category_Pretzels,Category_Pubs,Category_Ramen,Category_Recreation Centers,Category_Religious Organizations,Category_Restaurants,Category_Russian,Category_Salad,Category_Salvadoran,Category_Sandwiches,Category_Sc

In [ ]:
#useful code to view all columns of df

pd.set_option('display.max_columns', None)

In [143]:
backup_final_df = df

del df['categories']
del df['attributes']
del df['name']
del df['bizID']
del df['bizID-dt']
del df['lat']
del df['long']
del df['postal_code']
df['count_crucial_signficant']= df['count_crucial_signficant']>0



In [144]:
from nltk.corpus import stopwords
import nltk
import string
stop = stopwords.words('english')
df['review_text']=df['review_text'].str.lower()
df['review_text'] = df['review_text'].apply(lambda x: '  '.join([word for word in x.split() if word not in (stop)]))
ps = nltk.stem.WordNetLemmatizer()
df['review_text']=df["review_text"].apply(lambda x:[ps.lemmatize(y,pos='v') for y in x.split()])
df['review_text']=df['review_text'].apply(lambda x: ',  '.join(x))
translator = str.maketrans('', '', string.punctuation)
df['review_text'] = df['review_text'].str.translate(translator)

#df = df.convert_objects(convert_numeric=True)

In [145]:
X_ntext = df.drop(['review_text','count_crucial_signficant'], axis=1)

### Model Time!

In [148]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy.sparse
import sklearn.pipeline as pipe

#here we are creating a pipeline of the multiple steps needed to create n-grams and tfidf transformation
tfidf = pipe.Pipeline([
    ('vectorizer', HashingVectorizer(ngram_range=(1,4), non_negative=True)),
    ('tfidf', TfidfTransformer()),
])

#separate the non text features so that we only transform the text
X_text = df['review_text']
X_ntext = df.drop(['review_text','count_crucial_signficant'], axis=1)
#X_tfidf = tfidf.fit_transform(X_text)
vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))

# #combine the transformed sparse matrix with the non-text features
X = scipy.sparse.hstack((vectorizer.fit_transform(df.review_text), X_ntext.values))
Y = df['count_crucial_signficant']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.75)

In [150]:
model = LogisticRegression()
model.fit(X_train, Y_train)
print ("Area under the ROC curve on test data = %.3f" % metrics.roc_auc_score(model.predict(X_test), Y_test))
fpr, tpr, thresholds = metrics.roc_curve(Y_test, model.predict_proba(X_test)[:,1])



Area under the ROC curve on test data = 0.519
